# Deploying MMMs and CLVs in Production: Saving and Loading Models

Deploying Bayesian models written with PyMC has historically been a challenging process, with a myriad of details to consider. Yet, it's an essential part of the workflow that often has room for improvement.

In this publication, we delve into a new approach that tackles these deployment issues head-on. We introduce a new deployment module that, while not a silver bullet, brings about notable simplicity to the process.

This module marks a significant step towards improving the efficiency of deploying PyMC models. As we step into an era where ease of use and user-friendly solutions are at the forefront, let's explore how this module can make a tangible difference in your data science projects.


Recent release of PyMC-Marketing by Labs proves to be a big hit [check here <-](https://www.pymc-labs.io/blog-posts/pymc-marketing-a-bayesian-approach-to-marketing-data-science/). In the feedback one could see an ongoing theme, many of you have been requesting easy and robust way of deploying models to production. It’s been a long-standing problem with PyMC ( and most other PPLs). The reason for that is that there’s no obvious way, and doesn’t matter which approach you try it proves to be tricky. That is why we’re happy to announce the release of `ModelBuilder`, brand new PyMC-experimental module that addresses this need, and improves on the deployment process significantly.


For this notebook I'll use the example model used in [MMM Example Notebook](https://www.pymc-marketing.io/en/stable/notebooks/mmm/mmm_example.html), but ommit the details of data generation and plotting functionalities, since they're out of scope for this introduction, I highly recommend to see that part as well, but for now let's focus on today's topic: `ModelBuilder` and it's recent integration with PyMC-Marketing toolset.
The reason for choosing `DelayedSaturatedMMM` is that MMM classes make use of most of features provided in the project, including sklearn-like API

In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

from pymc_marketing.mmm.transformers import geometric_adstock, logistic_saturation
from pymc_marketing.mmm.delayed_saturated_mmm import DelayedSaturatedMMM

Let's load the dataset:

In [25]:
df = pd.read_csv('delayedSaturatedDataset.csv')
df

,Unnamed: 0,date_week,year,month,dayofyear,x1,x2,x1_adstock,x2_adstock,x1_adstock_saturated,x2_adstock_saturated,trend,cs,cc,seasonality,event_1,event_2,intercept,epsilon,y
0,0,2018-04-02,2018,4,92,0.318580,0.000000,0.191273,0.000000,0.364917,0.000000,0.778279,0.021487,-0.010744,0.005371,0.0,0.0,2.0,0.106260,3.984662
1,1,2018-04-09,2018,4,99,0.112388,0.000000,0.143987,0.000000,0.280268,0.000000,0.790638,0.259165,-0.130704,0.064231,0.0,0.0,2.0,0.067198,3.762872
2,2,2018-04-16,2018,4,106,0.292400,0.000000,0.233150,0.000000,0.435205,0.000000,0.802746,0.481904,-0.248773,0.116566,0.0,0.0,2.0,0.242042,4.466967
3,3,2018-04-23,2018,4,113,0.071399,0.000000,0.136127,0.000000,0.265721,0.000000,0.814614,0.676865,-0.363245,0.156810,0.0,0.0,2.0,0.095631,3.864219
4,4,2018-04-30,2018,4,120,0.386745,0.000000,0.286650,0.000000,0.517779,0.000000,0.826254,0.832810,-0.472462,0.180174,0.0,0.0,2.0,-0.118140,4.441625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,174,2021-08-02,2021,8,214,0.033024,0.000000,0.056235,0.001163,0.111998,0.001745,1.770035,-0.879312,-0.859141,-0.869226,0.0,0.0,2.0,0.313255,3.553546
175,175,2021-08-09,2021,8,221,0.165615,0.863349,0.121870,0.690914,0.239025,0.776451,1.773333,-0.967484,-0.791496,-0.879490,0.0,0.0,2.0,0.401690,5.565510
176,176,2021-08-16,2021,8,228,0.171882,0.000000,0.151917,0.138183,0.294818,0.204356,1.776620,-0.999887,-0.712405,-0.856146,0.0,0.0,2.0,-0.075989,4.137651
177,177,2021-08-23,2021,8,235,0.280257,0.000000,0.228954,0.027637,0.428377,0.041431,1.779895,-0.974654,-0.623010,-0.798832,0.0,0.0,2.0,0.129986,4.479041


But for our model we need much smaller dataset, many of the previous features were contributing to generation of others, now as our target variable is computed we can filter out not needed columns:

In [10]:
columns_to_keep = [
    "date_week",
    "y",
    "x1",
    "x2",
    "event_1",
    "event_2",
    "dayofyear",
]

data = df[columns_to_keep].copy()
# trend feature
data["t"] = range(n)
data.head()

,date_week,y,x1,x2,event_1,event_2,dayofyear,t
0,2018-04-02,3.984662,0.318580,0.0,0.0,0.0,92,0
1,2018-04-09,3.762872,0.112388,0.0,0.0,0.0,99,1
2,2018-04-16,4.466967,0.292400,0.0,0.0,0.0,106,2
3,2018-04-23,3.864219,0.071399,0.0,0.0,0.0,113,3
4,2018-04-30,4.441625,0.386745,0.0,0.0,0.0,120,4


## _Model Creation_
After we have our dataset ready, we could proceed straight to our model definition, but first to show the full potential of one of the new features of `ModelBuilder` - `model_config` we need to use some of our data to define our prior for sigma parameter for each of the channels. `model_config` is a customizable dictionary with keys corresponding to priors within the model, and values containing a dictionaries with parameters necessary to initialize them. Later on we'll learn that through the `save()` method we can preserve our priors contained inside the `model_config`, to allow complete replication of our model.

### model_config

First, let's compute the share of spend per channel:

In [11]:
total_spend_per_channel = data[["x1", "x2"]].sum(axis=0)

spend_share = total_spend_per_channel / total_spend_per_channel.sum()

spend_share

x1    0.65632
x2    0.34368
dtype: float64

Next, we specify the sigma parameter per channel:

In [12]:
# The scale necessary to make a HalfNormal distribution have unit variance
HALFNORMAL_SCALE = 1 / np.sqrt(1 - 2 / np.pi)

n_channels = 2

prior_sigma = HALFNORMAL_SCALE * n_channels * spend_share.to_numpy()

prior_sigma.tolist()

[2.1775326025486734, 1.1402608773919387]

`default_model_config` attribute of every model inheriting from `ModelBuilder` will allow you to see which priors are available for customization. To see it simply initialize a dummy model:

In [13]:
dummy_model = DelayedSaturatedMMM(date_column = '', channel_columns= '', adstock_max_lag = 4)
dummy_model.default_model_config

{'intercept': {'mu': 0, 'sigma': 2},
 'beta_channel': {'sigma': 2, 'dims': ('channel',)},
 'alpha': {'alpha': 1, 'beta': 3, 'dims': ('channel',)},
 'lam': {'alpha': 3, 'beta': 1, 'dims': ('channel',)},
 'sigma': {'sigma': 2},
 'gamma_control': {'mu': 0, 'sigma': 2, 'dims': ('control',)},
 'mu': {'dims': ('date',)},
 'likelihood': {'dims': ('date',)},
 'gamma_fourier': {'mu': 0, 'b': 1, 'dims': 'fourier_mode'}}

You can change only the prior parameters that you wish, no need to alter all of them, unless you'd like to!
In this case we'll just simply replace our sigma for beta_channel with our computed one:

In [14]:
custom_beta_channel_prior = {'beta_channel': {'sigma': prior_sigma, 'dims': ('channel',)}}
my_model_config = {**dummy_model.default_model_config, **custom_beta_channel_prior}

As mentioned in the original notebook: "_For the prior specification there is no right or wrong answer. It all depends on the data, the context and the assumptions you are willing to make. It is always recommended to do some prior predictive sampling and sensitivity analysis to check the impact of the priors on the posterior. We skip this here for the sake of simplicity. If you are not sure about specific priors, the `DelayedSaturatedMMM` class has some default priors that you can use as a starting point._"

The second feature that we can use for model definition is `sampler_config`. Similar to `model_config`, it's a dictionary that gets saved and contains things you'd usually pass to the `fit()` kwargs. It's not mandatory to create your own `sampler_config`; if not provided, both `model_config` and `sampler_config` will default to the forms specified by PyMC Labs experts, which allows for the usage of all model functionalities. The default `sampler_config` is left empty because the default sampling parameters usually prove sufficient for a start.

In [15]:
dummy_model.default_sampler_config

{}

In [16]:
my_sampler_config = {
    'tune':1000,
    'draws':1000,
}

Let's finally assemble our model!

In [17]:
mmm = DelayedSaturatedMMM(
    model_config = my_model_config,
    sampler_config = my_sampler_config,
    date_column="date_week",
    channel_columns=["x1", "x2"],
    control_columns=[
        "event_1",
        "event_2",
        "t",
    ],
    adstock_max_lag=8,
    yearly_seasonality=2,
)

An important thing to note here is that in the new version of `DelayedSaturatedMMM`, we don't pass our dataset to the class constructor itself. This is due to a reason I've mentioned before - it supports `scikit-learn` transformers and validations that require a usual X, y split and typically expect the data to be passed to the `fit()` method.

## _X, y Splitting:_

In [18]:
X = data.drop('y',axis=1)
y = data['y']

## _Model Fitting_

All that's left now is to finally fit the model:

In [19]:
mmm.fit(X=X, y=y, target_accept=0.95, chains=4, random_seed=rng)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [intercept, beta_channel, alpha, lam, sigma, gamma_control, gamma_fourier]
Traceback (most recent call last):
  File "/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/multiprocessing/forkserver.py", line 274, in main
    code = _serve_one(child_r, fds,
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/multiprocessing/forkserver.py", line 313, in _serve_one
    code = spawn._main(child_r, parent_sentinel)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/multiprocessing/spawn.py", line 130, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/michalraczycki/Documents/pymc-marketing/.conda/e

BrokenPipeError: [Errno 32] Broken pipe

The `fit()` method from `ModelBuilder` automatically builds the model using the priors from `model_config`, and assigns the created model to our instance. You can access it as a normal attribute.

In [ ]:
type(mmm.model)

In [ ]:
mmm.graphviz()

posterior trace can be accessed `fit_result` attribute

In [ ]:
mmm.fit_result

And if you wish to inspect the entire inference data, use `idata` attribute. Within `idata`, our entire dataset passed to the model can be found under `fit_data`

In [ ]:
mmm.idata

All the data passed to the model on initialisation is stored in `idata.attrs`. It will be later used in the `save()` method
to convert them, and all the fit data into netcdf format.

In [ ]:
mmm.idata.attrs

Simply specify the path to which you'd like to save your model:

In [ ]:
mmm.save('my_saved_model')

And pass it to the `load()` method when it's needed again on the target system:

In [ ]:
loaded_model = DelayedSaturatedMMM.load('my_saved_model')

In [ ]:
loaded_model.idata

In [ ]:
loaded_model.graphviz()

## Summary:

The `ModelBuilder` module is a new feature of `PyMC` based models. It provides 2 easy-to-use methods: `save()` and `load()` that can be used after the model has been fit.`save()` allow easy preservation of the model to .netcdf format, and `load()` gives one-line replication of the original model. Users can control the prior settings with `model_config`, and customize the sampling process using `sampler_config`.
Default values of those are working just fine, so first time give it a try without changing, and provide your own `model_config` and `model_sampler` if afterwards you want to try to customize it more for your use case!

Even though this introduction is about `DelayedSaturatedMMM`, functionalities from `ModelBuilder` are available in the CLV models as well.